*OBjective: Find where can El Salvador citizens travel without the need of a requesting a visa, and provide recommendations based on different factors of living conditions.* 

**data from:** 

***https://visaguide.world/visa-free-countries/salvadoran-passport/***

***https://www.worlddata.info/cost-of-living.php***


Notes: found a better more resourceful website:
https://www.numbeo.com/cost-of-living/

In [3]:
import requests
import pandas as pd
import numpy as np
import csv
from bs4 import BeautifulSoup

In [16]:
r = requests.get('https://www.worlddata.info/cost-of-living.php')
soup = BeautifulSoup(r.text, 'html.parser')

world_data = [tag for tag in soup.find_all(['tr'])]
country_bucket = []
column_names = ['Countries'] # scraped country names from https://www.worlddata.info/cost-of-living.php

n = 1
while n < len(world_data):
    countries = world_data[n].a.text
    n+=1
    country_bucket.append(countries)

country_bucket = sorted(country_bucket)

df = pd.DataFrame(country_bucket, columns=column_names)
df

,Countries
0,Albania
1,Algeria
2,Australia
3,Austria
4,Azerbaijan
...,...
95,United Kingdom
96,United States
97,Uzbekistan
98,Vanuatu


In [34]:
r = requests.get('https://www.numbeo.com/quality-of-life/rankings_by_country.jsp')
soup = BeautifulSoup(r.text, 'html.parser')

data_for_column_names = [tag for tag in soup.find_all(['th'])]


In [49]:
column_names2 = []

o = 0
while o < len(data_for_column_names):
    column_data = data_for_column_names[o].text
    o+=1
    column_names2.append(column_data)
    
column_names2

['Rank',
 'Country',
 'Quality of Life Index',
 'Purchasing Power Index',
 'Safety Index',
 'Health Care Index',
 'Cost of Living Index',
 'Property Price to Income Ratio',
 'Traffic Commute Time Index',
 'Pollution Index',
 'Climate Index']